In [1]:
import pandas as pd

1. Просмотр содержимого датасетов

In [2]:
df_marketing = pd.read_csv('data/marketing_campaign.csv', sep=',')
df_marketing.head(3)

,user_id,date_served,marketing_channel,variant,language_displayed,converted
0,a1000,1/1/18,House Ads,personalization,English,True
1,a1001,1/1/18,House Ads,personalization,English,True
2,a1002,1/1/18,House Ads,personalization,English,True


In [3]:
df_subscribers = pd.read_csv('data/subscribers.csv', sep=',')
df_subscribers.head(3)

,user_id,subscribing_channel,date_subscribed,date_canceled,is_retained
0,a1000,House Ads,1/1/18,NaN,True
1,a1001,House Ads,1/1/18,NaN,True
2,a1002,House Ads,1/1/18,NaN,True


In [4]:
df_users = pd.read_csv('data/users.csv', sep=',')
df_users.head(3)

,user_id,age_group,language_preferred
0,a1000,0-18 years,English
1,a1001,19-24 years,English
2,a1002,24-30 years,English


In [5]:
df = df_users.merge(df_marketing, on='user_id', how='inner')
df = df.merge(df_subscribers, on='user_id', how='inner')
df.head()

,user_id,age_group,language_preferred,date_served,marketing_channel,variant,language_displayed,converted,subscribing_channel,date_subscribed,date_canceled,is_retained
0,a1000,0-18 years,English,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True
1,a1001,19-24 years,English,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True
2,a1002,24-30 years,English,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True
3,a1003,30-36 years,English,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True
4,a1004,36-45 years,English,1/1/18,House Ads,personalization,English,True,House Ads,1/1/18,NaN,True


2. Типы и статистики колонок

In [6]:
df.dtypes

user_id                object
age_group              object
language_preferred     object
date_served            object
marketing_channel      object
variant                object
language_displayed     object
converted              object
subscribing_channel    object
date_subscribed        object
date_canceled          object
is_retained            object
dtype: object

In [7]:
df.describe()

,user_id,age_group,language_preferred,date_served,marketing_channel,variant,language_displayed,converted,subscribing_channel,date_subscribed,date_canceled,is_retained
count,10037,10037,10037,10021,10022,10037,10037,10022,1856,1856,577,1856
unique,10037,7,4,31,5,2,4,2,5,31,115,2
top,a1000,19-24 years,English,1/15/18,House Ads,control,English,False,Instagram,1/16/18,4/2/18,True
freq,1,1682,9275,789,4733,5091,9793,8946,600,163,15,1279


3. Определить эффективность маркетинговых каналов по привлечению платящих игроков

In [8]:
pd.crosstab(
    index=df['marketing_channel'],
    columns=df['converted'],
    normalize='index'
)

converted,False,True
marketing_channel,,
Email,0.658407,0.341593
Facebook,0.872581,0.127419
House Ads,0.937038,0.062962
Instagram,0.858365,0.141635
Push,0.916415,0.083585


4. Определить количество игроков в каждой возрастной группе

In [9]:
df.groupby(by='age_group', sort=True)['user_id'].count()

age_group
0-18 years     1539
19-24 years    1682
24-30 years    1568
30-36 years    1355
36-45 years    1353
45-55 years    1353
55+ years      1187
Name: user_id, dtype: int64

5. Определить самую раннюю дату подписки на сервис

In [10]:
df['date_subscribed'].sort_values().head(1)

0    1/1/18
Name: date_subscribed, dtype: object

6. Определить портрет аудитории удержанных подписчиков (по возрасту и языку)

In [11]:
keep_subs = df[df['is_retained'].eq(True)]
keep_t = pd.crosstab(
    index=keep_subs['age_group'],
    columns=keep_subs['language_preferred'],
    normalize='all'
)

keep_t

language_preferred,Arabic,English,German,Spanish
age_group,,,,
0-18 years,0.002346,0.164191,0.007819,0.004691
19-24 years,0.000782,0.237686,0.008600,0.007819
24-30 years,0.002346,0.195465,0.011728,0.003909
30-36 years,0.000000,0.089132,0.000782,0.005473
36-45 years,0.000782,0.083659,0.001564,0.003909
45-55 years,0.001564,0.084441,0.002346,0.004691
55+ years,0.001564,0.067240,0.001564,0.003909


In [12]:
keep_t.idxmax()

language_preferred
Arabic      0-18 years
English    19-24 years
German     24-30 years
Spanish    19-24 years
dtype: object